ERROR! Session/line number was not unique in database. History logging moved to new session 383



KeyboardInterrupt



In [5]:
import angr
import re
from angr.knowledge_plugins.cfg import CFGNode
import matplotlib.pyplot as plt
import networkx as nx

def adjlist_to_dict(adjlist):
    graph = {}
    pattern = r'<[^>]+>'  # Matches anything enclosed in <...>

    for line in adjlist:
        nodes = re.findall(pattern, line)  # Extract all nodes in the line
        if nodes:
            node = nodes[0]
            neighbors = nodes[1:]
            graph[node] = neighbors

    return graph

proj = angr.Project('/Users/jglez2330/Library/Mobile Documents/com~apple~CloudDocs/personal/STARK-attesttation/test/a.out', load_options={'auto_load_libs': False})
cfg = proj.analyses.CFGFast(normalize=True, show_progressbar=True, resolve_indirect_jumps=True)

#end_state, cfg, path = get_execution_path(proj, cfg) # get example execution path
#print("Execution path ended because it %s" % end_state)

graph = cfg.graph
adj = nx.generate_adjlist(graph)
adj = adjlist_to_dict(adj)
print(adj) 

WARNING  | 2024-10-23 22:01:45,899 | cle.backends.macho.macho | The Mach-O backend is not well-supported. Good luck!
WARNING  | 2024-10-23 22:01:45,902 | cle.backends.macho.macho | 14 is not handled yet
WARNING  | 2024-10-23 22:01:45,902 | cle.backends.macho.macho | 27 is not handled yet
WARNING  | 2024-10-23 22:01:45,903 | cle.backends.macho.macho | 42 is not handled yet


Output()

{'<CFGNode _main [29]>': ['<CFGNode 0x100000f52[6]>', '<CFGNode _main+0x1d [18]>'], '<CFGNode 0x100000f52[6]>': ['<CFGNode UnresolvableJumpTarget [0]>'], '<CFGNode UnresolvableJumpTarget [0]>': [], '<CFGNode _main+0x1d [18]>': ['<CFGNode 0x100000f58[6]>', '<CFGNode _main+0x2f [20]>'], '<CFGNode 0x100000f58[6]>': ['<CFGNode UnresolvableJumpTarget [0]>'], '<CFGNode _main+0x2f [20]>': ['<CFGNode _main+0x43 [17]>', '<CFGNode _main+0x59 [17]>'], '<CFGNode _main+0x43 [17]>': ['<CFGNode 0x100000f52[6]>', '<CFGNode _main+0x54 [5]>'], '<CFGNode _main+0x59 [17]>': ['<CFGNode 0x100000f52[6]>', '<CFGNode _main+0x6a [8]>'], '<CFGNode _main+0x6a [8]>': [], '<CFGNode _main+0x54 [5]>': ['<CFGNode _main+0x6a [8]>'], '<CFGNode 0x100000f6f[9]>': [], '<CFGNode 0x100000fa4[30]>': ['<CFGNode 0x100000fd0[44]>', '<CFGNode 0x100000fc2[11]>'], '<CFGNode 0x100000fd0[44]>': [], '<CFGNode 0x100000fc2[11]>': []}


In [11]:
import random

def all_reachable_paths(graph):
    """Precompute all reachable nodes for each starting node."""
    reachable = {}

    def dfs(node, visited=None):
        if visited is None:
            visited = set()
        visited.add(node)

        for neighbor in graph.get(node, []):
            if neighbor not in visited:
                dfs(neighbor, visited)
        return visited

    for node in graph:
        reachable[node] = dfs(node)

    return reachable

def find_path(graph, start, end, path=None):
    """DFS to find a path from start to end."""
    if path is None:
        path = []

    path = path + [start]

    if start == end:
        return path

    if start not in graph:
        return None

    for neighbor in graph[start]:
        if neighbor not in path:
            new_path = find_path(graph, neighbor, end, path)
            if new_path:
                return new_path

    return None

def random_path_with_reachable(graph):
    """Select two nodes with a valid path between them."""
    reachable = all_reachable_paths(graph)

    # Collect pairs (start, end) where a path exists
    valid_pairs = [
        (start, end)
        for start, reachable_nodes in reachable.items()
        for end in reachable_nodes
        if start != end  # Ensure they are distinct
    ]

    if not valid_pairs:
        print("No valid paths found in the graph.")
        return None

    # Pick a random valid pair
    start, end = random.choice(valid_pairs)
    print(f"Finding path from {start} to {end}...")

    # Find and print the path
    path = find_path(graph, start, end)

    if path:
        print(f"Path found: {path}")
        return path
    else:
        print(f"No path found from {start} to {end}.")
        return None
path = random_path_with_reachable(adj)
path

Finding path from <CFGNode _main+0x43 [17]> to <CFGNode _main+0x54 [5]>...
Path found: ['<CFGNode _main+0x43 [17]>', '<CFGNode _main+0x54 [5]>']


['<CFGNode _main+0x43 [17]>', '<CFGNode _main+0x54 [5]>']

In [13]:
from hashlib import sha256

Path found: A -> C -> E -> F


['A', 'C', 'E', 'F']

In [61]:
#execution = [sha256(i.encode("UTF-8")) for i in path]
import hash
#path = [str(i) for i in path]
execution = []
for i in path:
    execution.append(i.encode("UTF-8"))
trace = []
for i in execution:
    print(int.from_bytes(i))
    trace.append(int.from_bytes(i))
    
nonce = 1

trace.append(nonce)

trace = hash.poseidon_hash(trace)

#trace.append(sha256(str(nonce).encode("UTF-8")))
#for i in range(len(execution)-1):
#    trace.append(sha256(execution[i].digest() + trace[-1].digest()))
#trace = [int.from_bytes(i.digest()) for i in trace]

trace

378275670689887191647802758758334741584922576964119267073342
1477639338632371842374229526399745084316104098840579497278


[6048755700949961189669124726068799266882965611467975912202410078793260939162,
 520659857297231285359286884718005448430239945804127534760094834127155511309,
 11424006874823228227490993830540773901986650769882319476818743576143762898379]

In [62]:
import math
def next_power_of_2(n):
    if n < 1:
        return 1
    return 2 ** math.ceil(math.log2(n))

In [63]:
from Field import  *
from Polynomial import  *
exp_factor = 8
root_pow = next_power_of_2(len(trace))
field = Field.main()
g = field.primitive_nth_root(root_pow)
h = field.primitive_nth_root(root_pow*exp_factor)

G = [g^i for i in range(len(trace))]
H = [h^i for i in range(root_pow*exp_factor)]


In [64]:
trace = [FieldElement(i, Field.main()) for i in trace]

In [65]:
fx = Polynomial.interpolate_domain(G, trace)
fx.degree()

2

In [66]:
random_w = FieldElement(4, Field.main())
H_w = [h*random_w for h in H]
fx_exp_eval = fx.evaluate_domain(H_w)
H_w


In [67]:
from Merkle import *
print(len(fx_exp_eval))
fx_root =  MerkleTree(fx_exp_eval)
fx_root

32


# Contrains
3 constrains
final and initial value must be the same
and intermidiate value must be the following formula:
$a_n = a_{n-1} + m$
where m is the slope of the line

In [71]:
nonce_hash = FieldElement(6048755700949961189669124726068799266882965611467975912202410078793260939162, Field.main())
p0_num = fx - Polynomial([nonce_hash])
x = Polynomial([field.zero(), field.one()])
p0_dem = x - Polynomial([G[0]])
p0 = p0_num/p0_dem
p0

In [72]:
p1_num = fx - Polynomial([FieldElement(11424006874823228227490993830540773901986650769882319476818743576143762898379, Field.main())])
x = Polynomial([field.zero(), field.one()])
p1_dem = x - Polynomial([g^(len(trace)-1)])
p1 = p1_num/p1_dem
p1

In [73]:
cp = p0 + p1
cp_eval = cp.evaluate_domain(H_w)
for i in cp_eval:
    print(i.value)
mk = MerkleTree(cp_eval)

261016264982363350970505146787942169964
124887616140903457227299320574249172113
101377520561277043676503658507476880677
248465489985511146169841300882821702590
63402008705076711447106503958250635527
17559599429412069765206484911335693036
14132089957680393538024890239965026083
243811054572534080746458876914812367764
18998597405001675148735797473254087310
260161884108750835672791563211688014988
59231302124177282705185188869766394571
82288962122318329443456178006866501085
217341759307266530891002427438562842892
67329986743641643373422955688383825636
87002396261612991295814952879908774796
173224724736674793995895023803183558246
251403010374058040999303543453680027126
117033762073287554606584632900322903760
140543857652913968157380294967095195196
263953785370910245799967389358800494500
178519369509114300386777449516321440346
224361778784778942068677468563236382837
227789288256510618295859063234607049790
268608220783887311223349813326809829326
222922780809189336685148156001317988563
25225739

# FRI

In [74]:
from FRI import *

fri = Fri(H_w)

fri_polys, fri_domains, fri_layers, fri_merkles = fri.commit(cp, H_w, cp_eval, mk, None)

next_len fri layer: 16


In [75]:
#fri.decommit(1, fri_polys, fri_domains, fri_layers, fri_merkles)
[len(i) for i in fri_layers]

[32, 16]

# Verify




In [76]:
mt_root = fx_root 

In [77]:
# 3 cp so 3 alphas
alpha0, alpha1,alpha2 = FieldElement(1, Field.main()), FieldElement(1, Field.main()), FieldElement(1, Field.main())

In [78]:
# CP merkle root
cp_root = mk

In [79]:
idx = 1

In [80]:
proof_f = fri.decommit_on_query(idx, fx_exp_eval, mt_root)

In [81]:
proof_cp = fri.decommit_on_fri(idx, fri_polys, fri_domains, fri_layers, fri_merkles)


In [82]:
# Basic check
assert len(proof_cp) % 2
v_last_one = proof_cp.pop()

In [83]:
last_one = fri_polys[-1].coefficients[0] # poly 0 shall be constant

v_last_one
assert v_last_one == last_one


In [84]:
from Merkle import verify_decommitment

In [85]:
# Check polynomial constraint
# First check evaluation of polynomial f is honest
v_f_eval = proof_f[0::2]
v_f_auth = proof_f[1::2]
assert len(v_f_eval) == len(v_f_auth)
for i in range(len(v_f_eval)):
    assert verify_decommitment(idx + 8 * i, v_f_eval[i], v_f_auth[i], mt_root.root ), f'in iter {i}, go wrong'
# Then check whether they could satify the recursive condition
print(proof_cp)
v_cp0 = proof_cp[:2]
length = len(fri_layers[0]) # 8192 shoud be a prior knowledge
assert verify_decommitment(idx % length, v_cp0[0], v_cp0[1], cp_root.root)

48585b4b7accd35d6a6b1ec3c0bfba98b8c08ef1d0615227d5c54cfbc74206f5
48585b4b7accd35d6a6b1ec3c0bfba98b8c08ef1d0615227d5c54cfbc74206f5
[<Field.FieldElement object at 0x1185ce720>, ['cce6f855a7851c1717d212a81d1f14555a34ce6e6c79e45c6ff0b4a865e357de', 'd632b1e33e7f19a1c1e78b98d393d1efb2bf5bd3cdcb61198042a1e75850d774', '7f133e06c25584d144df98d21ab83fa91ba0e3c9ec4176c5b0eb95f65175e09c', '55dd59e2f04d059d1d4f8f07dd33341e16527e8198f99841569026ca5b4baf9f', '84df777bdfb59104a329ad265a88361adadd41bd188e5b8a0d099419857bf8a6'], <Field.FieldElement object at 0x11849ecc0>, ['5d84c435b47c75f5e777c088a7a5b7bd23b419bb92012fbc9ccf9939d3ca956f', 'a17e29909a29ee4ce896e6d9404ddc2a73b015b0a4feedb8a8148ae4f67eef12', '18747a698d4449bdd96626d37eb9b4a72e3d4589cdb92a1dec79d9c9128b7ee4', 'ab9e0312215e68d9f391dd0a119b7ced887fcf0acb55ba5ecdbcfecc97849ab3', 'fbde7662f6a595f4e884c8f856dd1ec8e0b2122727f2bbf6e47fd9059d41020d']]
0ea117f058b747c5d23ce2ae066e1089fdbc14f9664faaa709731b8bec3fa68c


In [87]:
# Check the polynomial constraint
fx = v_f_eval[0]
fgx = v_f_eval[1]
x = H_w[idx]
p0 = (fx - nonce_hash) / (x - G[0])
p1 = (fx - FieldElement(11424006874823228227490993830540773901986650769882319476818743576143762898379, Field.main()))/ (x - G[len(trace)-1])

#p2 = p2_num/p2_dem
assert v_cp0[0] == (p0 + p1)
print("poly constraint success!")

poly constraint success!


In [88]:
# Check low degree
v_cp = proof_cp[::2]
v_auth = proof_cp[1::2]
assert len(v_cp) == len(v_auth)
assert len(v_cp) % 2 == 0
k = length
for i in range(len(v_cp)//2 -1):
    iter_idx = idx % k
    iter_sib_idx = (idx + k // 2) % k
    assert verify_decommitment(iter_idx, v_cp[2*i], v_auth[2*i], fri_merkles[i].root)
    assert verify_decommitment(iter_sib_idx, v_cp[(2*i) + 1], v_auth[(2*i)+1], fri_merkles[i].root)
    k = k // 2

In [89]:
k = length
x = H_w[idx]
beta = [FieldElement(1, Field.main()) for i in range(99)]
for i in range(len(v_cp)//2 - 1):
    op1 = (v_cp[2*i] + v_cp[2*i + 1]) / FieldElement(2, Field.main())
    op2 = (v_cp[2*i] - v_cp[2*i + 1]) / (FieldElement(2, Field.main())*x)
    rhs = op1 + beta[i] * op2
    assert v_cp[2*(i+1)] == rhs, f" round {i}, CP(i+1) is {v_cp[2*(i+1)]} while rhs is {rhs}"
    x = x^2
print("low degree test success!")

low degree test success!
